<a href="https://colab.research.google.com/github/dkbs12/External_test/blob/main/extractiveQA_modulization_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%bash
pip install --upgrade pip
pip install farm-haystack[colab,elasticsearch,inference,ocr,preprocessing,file-conversion,pdf]
wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 70.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 7.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 49.7 MB/s eta 0:00:00
  Prepari

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.


In [ ]:
%%bash
pip install --upgrade pip
pip install farm-haystack[colab,elasticsearch,inference,ocr,preprocessing,file-conversion,pdf]

In [ ]:
%%bash --bg
sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import os
import time
from haystack.document_stores.elasticsearch import ElasticsearchDocumentStore
from haystack.utils import fetch_archive_from_http, convert_files_to_docs
from haystack.nodes import PreProcessor
from haystack.nodes import BM25Retriever, EmbeddingRetriever, FARMReader
from haystack.utils import print_answers
from haystack.pipelines import Pipeline
from haystack.nodes import JoinDocuments

In [ ]:
time.sleep(30)
host = os.environ.get("ELASTICSEARCH_HOST", "localhost")
document_store = ElasticsearchDocumentStore(host=host, username="", password="", index="document")

In [ ]:
def processor():
  doc_dir = "data/Phase1_test_data"
  url = "https://github.com/dkbs12/External_test/raw/main/Phase1_test_data.zip"
  fetch_archive_from_http(url=url, output_dir=doc_dir)
  got_docs = convert_files_to_docs(dir_path=doc_dir)
  preprocessor = PreProcessor(
    clean_whitespace=True,
    clean_header_footer=True,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=True,)
  all_docs = preprocessor.process(got_docs)
  document_store.delete_documents()
  document_store.write_documents(all_docs)
  bm25_retriever = BM25Retriever(document_store=document_store)
  embedding_retriever = EmbeddingRetriever(
  document_store=document_store, embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1"
  )
  document_store.update_embeddings(embedding_retriever, update_existing_embeddings=False)
  reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
  return bm25_retriever, embedding_retriever, reader

In [ ]:
def pipeline(query):
  bm25_retriever, embedding_retriever, reader = processor()
  p_ensemble = Pipeline()
  p_ensemble.add_node(component=bm25_retriever, name="BM25Retriever", inputs=["Query"])
  p_ensemble.add_node(component=embedding_retriever, name="EmbeddingRetriever", inputs=["Query"])
  p_ensemble.add_node(
  component=JoinDocuments(join_mode="concatenate"), name="JoinResults", inputs=["BM25Retriever", "EmbeddingRetriever"]
  )
  p_ensemble.add_node(component=reader, name="Reader", inputs=["JoinResults"])
  res = p_ensemble.run(
    query=query, params={"EmbeddingRetriever": {"top_k": 5}, "BM25Retriever": {"top_k": 5}, "Reader":{"top_k": 5}}
  )
  return res

In [ ]:
if __name__ == '__main__':
  query = 'What is NDC'
  res = pipeline(query)
  print_answers(res, details="medium")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
Preprocessing: 100%|██████████| 3/3 [00:00<00:00, 31.80docs/s]


(…)e/main/config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

(…)70bdf8fca0ca826b6b5d16ebc/.gitattributes:   0%|          | 0.00/737 [00:00<?, ?B/s]

(…)ca0ca826b6b5d16ebc/1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

(…)abd4f70bdf8fca0ca826b6b5d16ebc/README.md:   0%|          | 0.00/8.65k [00:00<?, ?B/s]

...

(…)erta-base-squad2/resolve/main/merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

(…)ad2/resolve/main/special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  3.29 Batches/s]

'Query: What is NDC'
'Answers:'
[   {   'answer': 'an XML-based communication standard',
        'context': 'is NDC?\n'
                   'New Distribution Capability or NDC is essentially an '
                   'XML-based communication standard created by the '
                   'International Air Transportation Associat',
        'score': 0.6344707608222961},
    {   'answer': 'allows carriers to take back control of their products '
                  'adjusting offers to current market demand and customer '
                  'preferences',
        'context': 'e NDC approach allows carriers to take back control of '
                   'their products adjusting offers to current market demand '
                   'and customer preferences. Airlines can',
        'score': 0.4891771674156189},
    {   'answer': 'airlines\n'
                  'avoid the two-step process of fare ﬁling, and availability\n'
                  'computation and distribution',
        'context': 'With NDC, a

In [ ]:
if __name__ == '__main__':
  query = 'Who is Dr. Belobaba?'
  res = pipeline(query)
  print_answers(res, details="medium")

Updating embeddings:   0%|          | 0/103 [00:00<?, ? Docs/s]

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

Updating embeddings: 10000 Docs [00:03, 2815.25 Docs/s]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Inferencing Samples: 100%|██████████| 1/1 [00:00<00:00,  5.33 Batches/s]


'Query: Who is Dr. Belobaba?'
'Answers:'
[   {   'answer': 'if demand is reaching capacity or is greater than capacity',
        'context': 'o Dr. Belobaba, continuous pricing works best if demand is '
                   'reaching capacity or is greater than capacity. As airlines '
                   'prepare for the post-COVID recov',
        'score': 0.8851358890533447},
    {   'answer': 'Principal Research Scientist',
        'context': 'namic offer generation and\n'
                   'continuous pricing, Dr. Belobaba, Principal Research '
                   'Scientist in the International Center for Air\n'
                   'Transportation at MIT an',
        'score': 0.6438376307487488},
    {   'answer': 'Principal Research Scientist',
        'context': 'n Massachusetts Institute of Technology\n'
                   'Peter P. Belobaba is Principal Research Scientist in the '
                   'International Center for Air Transportation at the Ma',
        'score': 0.55602294